# Imports

In [1]:
import torch
import pytorch_lightning as pl
import tensorboard as tb
import json
import time
import warnings
from torch import nn
from pytorch_lightning.accelerators import *
from torch.utils.tensorboard import SummaryWriter
from lightning.pytorch.accelerators import find_usable_cuda_devices
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.data.encoders import GroupNormalizer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, DeviceStatsMonitor
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from dataloading_helpers import electricity_dataloader
from config import *

# EDA

In the following we will take a short look into the electricity dataset.
To not create bias, we will just look at the train dataset.
Because the dataset is not too complex, we will mostly look at the distribution of the target value.

In [2]:
train, _, _ = electricity_dataloader.create_electricity_dataframe_not_normalized()  
print(train.info())

In [8]:
train.head()

,power_usage,time_idx,days_from_start,categorical_id,date,id,hour,day,day_of_week,month,hours_from_start,categorical_day_of_week,categorical_hour
17544,2.538071,26304,1096,MT_001,2014-01-01 00:00:00,MT_001,0,1,2,1,26304.0,2,0
17545,2.855330,26305,1096,MT_001,2014-01-01 01:00:00,MT_001,1,1,2,1,26305.0,2,1
17546,2.855330,26306,1096,MT_001,2014-01-01 02:00:00,MT_001,2,1,2,1,26306.0,2,2
17547,2.855330,26307,1096,MT_001,2014-01-01 03:00:00,MT_001,3,1,2,1,26307.0,2,3
17548,2.538071,26308,1096,MT_001,2014-01-01 04:00:00,MT_001,4,1,2,1,26308.0,2,4


In [9]:
train.power_usage.unique()

array([2.53807107e+00, 2.85532995e+00, 3.17258883e+00, ...,
       1.78918919e+04, 2.08783784e+04, 1.94459459e+04])

In [5]:
import seaborn as sns
sns.displot(train, x="flipper_length_mm")

In [ ]:
print("Importing modules...")
import torch
import pytorch_lightning as pl
import tensorboard as tb
import json
import time
import warnings
from torch import nn
from pytorch_lightning.accelerators import *
from torch.utils.tensorboard import SummaryWriter
from lightning.pytorch.accelerators import find_usable_cuda_devices
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.data.encoders import GroupNormalizer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, DeviceStatsMonitor
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from dataloading_helpers import electricity_dataloader
from config import *
import pickle
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

print("Preparing dataset...") 
  
electricity = electricity_dataloader.create_electricity_timeseries_tft()
timeseries_dict =  electricity
config_name_string = "electricity"
parameters = []
model_dir = CONFIG_DICT["models"][config_name_string]



  # create study
study = optimize_hyperparameters(
    electricity["train_dataloader"],
    electricity["val_dataloader"],
    model_path="hyperparameters_electricity",
    n_trials=100,
    max_epochs=20,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(16, 256),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.0005, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=40, max_epochs=20, log_every_n_steps=5),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=True,  # use Optuna to find ideal learning rate or use in-built learning rate finder
    optimizer="adam"
  )

  # save study results - also we can resume tuning at a later point in time
with open("test_study.pkl", "wb") as fout:
    pickle.dump(study, fout)

# show best hyperparameters
print(study.best_trial.params)

Importing modules...
Preparing dataset...


[I 2023-03-04 14:31:09,151] A new study created in memory with name: no-name-2de26a1c-68a9-49a7-8d22-474de4bdc72e
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Benedikt\anaconda_main\lib\site-packages\pytorch_lightning\utilities\parsing.py:262: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
C:\Users\Benedikt\anaconda_main\lib\site-packages\pytorch_lightning\utilities\parsing.py:262: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU av

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Restoring states from the checkpoint path at C:\Users\Benedikt\Desktop\Mein_Ordner\WI-INFO\Semester_4\Information_Systems\TFT_project\RT1_TFT\.lr_find_8adb9c78-ec14-4253-ad54-6616c2e6a44f.ckpt
Restored all states from the checkpoint file at C:\Users\Benedikt\Desktop\Mein_Ordner\WI-INFO\Semester_4\Information_Systems\TFT_project\RT1_TFT\.lr_find_8adb9c78-ec14-4253-ad54-6616c2e6a44f.ckpt
[I 2023-03-04 14:33:21,639] Using learning rate of 0.00198
C:\Users\Benedikt\anaconda_main\lib\site-packages\optuna\integration\pytorch_lightning.py:52: UserWarning: The metric 'val_loss' is not in the evaluation logs for pruning. Please make sure you set the correct metric name.
  warnings.warn(message)
`Trainer.fit` stopped: `max_epochs=20` reached.
[I 2023-03-04 15:05:54,553] Trial 0 finished with value: 0.28909581899642944 and parameters: {'gradient_clip_val': 0.3212525138750618, 'hidden_size': 16, 'dropout': 0.10785436359834565, 'hidden_continuous_size

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Restoring states from the checkpoint path at C:\Users\Benedikt\Desktop\Mein_Ordner\WI-INFO\Semester_4\Information_Systems\TFT_project\RT1_TFT\.lr_find_4a6b8276-4874-47b0-a010-a13b22d7c48e.ckpt
Restored all states from the checkpoint file at C:\Users\Benedikt\Desktop\Mein_Ordner\WI-INFO\Semester_4\Information_Systems\TFT_project\RT1_TFT\.lr_find_4a6b8276-4874-47b0-a010-a13b22d7c48e.ckpt
[I 2023-03-04 15:16:48,595] Using learning rate of 0.000896
C:\Users\Benedikt\anaconda_main\lib\site-packages\optuna\integration\pytorch_lightning.py:52: UserWarning: The metric 'val_loss' is not in the evaluation logs for pruning. Please make sure you set the correct metric name.
  warnings.warn(message)
